<a href="https://colab.research.google.com/github/bwsi-hadr/03-Intro-to-GIS/blob/master/03_Intro_to_GIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to GIS with Python
## What is GIS?
GIS stands for _geographic information system_. Colloquially, it's the process of presenting and analyzing data on maps. GIS allows us to visualize and characterize the nature of spatially distributed data, including weather, infrastructure, and populations. As you can imagine, this is key for disaster response scenarios for both diagnosing the situation, as well as planning and monitoring the response.

There are dozens of different GIS software options, both free and commercial. In this course, we will focus on free, python-based tools and packages. The principles taught in this course should carry over to most common GIS implementations. 

In particular, we will be using:
- GDAL
- geopandas

This content is based off of the [Automating GIS Processes course](https://automating-gis-processes.github.io/2018/) from the University of Helsinki

In [ ]:
try: # try the following block of code
  import geopandas as gpd
except: # if the try block throws an error, run the following
  !pip install geopandas # install geopandas
  import geopandas as gpd
  
try:
  import contextily as ctx # import
except:
  !apt install libproj-dev proj-data proj-bin
  !apt install libgeos-dev
  !pip install cython
  !pip install cartopy
  !pip install contextily==1.0rc1 --no-use-pep517 --no-cache-dir
import contextily as ctx # library for providing background maps
import fiona # library for reading/writing GIS files, comes w/ geopandas
from shapely.geometry import Point, LineString, Polygon
from matplotlib import pyplot as plt

## Reading in GIS data
For this lesson we are using data in Shapefile format representing distributions of specific beautifully colored fish species called Damselfish and the country borders of Europe.

We're going to use the `wget` terminal command to download a file from a url.
We then use `unzip` to unzip the archive into a folder of the same name. The `-o` option is used to overwrite the folder if it already exists
We then us `ls` to see the contents of the folder

In [ ]:
!wget https://github.com/Automating-GIS-processes/Lesson-2-Geo-DataFrames/raw/master/data/Data.zip -O fish_data.zip
!unzip -o fish_data.zip 
!ls Data

Typically reading the data into Python is the first step of the analysis pipeline. In GIS, there exists various dataformats such as [Shapefile](https://en.wikipedia.org/wiki/Shapefile), [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), [KML](https://en.wikipedia.org/wiki/Keyhole_Markup_Language), and [GPKG](https://en.wikipedia.org/wiki/GeoPackage) that are probably the most common vector data formats. Geopandas is capable of reading data from all of these formats (plus many more). Reading spatial data can be done easily with geopandas using `gpd.from_file()` -function:

In [ ]:
# path to shapefile
filepath = "Data/DAMSELFISH_distributions.shp"

# Read file using gpd.read_file()
data = gpd.read_file(filepath)

In [ ]:
data.head() #look at top entries - looks like a pandas dataframe

In [ ]:
# Note the column 'geometry' is full of shapely Polygon objects
type(data['geometry'].iloc[0])

In [ ]:
# geopandas adds useful attributes to the geodataframe, such as the ability to get bounds
# of all the geometry data
data.bounds

In [ ]:
# similary, we can get attributes such as boundary
data.boundary

In [ ]:
# area
data.area

In [ ]:
# we can check which species can be found between latitudes 10 and 20 degrees north
data.intersects(Polygon([(-180,10),(180,10),(180,20),(-180,20)]))

## Exercises
Using the polygon objects in the `geometry` column of the data frame:
- create a new column called `area` which represent the areas of each row in the shapefile
- What are the max, min, median, and quartiles values of the areas?
- What fraction of the areas are greater than 25 square degrees?
- What species has the largest total area?

## Plotting
Geopandas provides a useful `.plot()`  function which creates a matplotlib figure and returns an axes object.

There's a ton of additional libraries that provide more plotting functionality, and we'll explore a few of them here. There's no "correct" set of libraries to use for GIS in python, and it's up to you to figure out which ones fit the best into your workflow.

The `cmap` option to the `.plot()` function allows you to pass in a [matplotlib colormap name](https://matplotlib.org/gallery/color/colormap_reference.html), which are collections of colors used to visualize data

In [ ]:
# we can use the built-in geopandas plot function to visualize
ax = data.plot(figsize=(10,5), alpha=0.6, cmap='Set2')

currently the colors are assigned arbitrarily. However, we can also use colors to encode information. 

Let's first use colors to categorize by endangerment status. To do so, we pass the `column` argument to `plot()`. For reference, we also set `legend=True`

In [ ]:
ax = data.plot(figsize=(10,5), alpha=0.6, cmap='Set2', column='CATEGORY', legend=True)

Another common use of colors to encode data is to represent numerical data in an area with colors. This is known as a [choropleth](https://en.wikipedia.org/wiki/Choropleth_map).

Let's use this to encode the areas of each region

In [ ]:
#first create a column of areas
data['area'] = data.area

#then pass the area column as an argument
ax = data.plot(figsize=(10,5), alpha=0.6, column='area', legend=True)

The colorbar legend is too big relative to the figure. We'll have to do some manual adjustments. There are tools to create axes grids for colorbars available in:
https://matplotlib.org/3.1.0/tutorials/toolkits/axes_grid.html

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(1, 1)
divider = make_axes_locatable(ax) #makes it so you can append to the axes

#put another axes to the right of it, at 5% of the total width with 0.1 points of padding in between
cax = divider.append_axes("right", size="5%", pad=0.1) 
# note that you have to specify both ax and cax as arguments for it to work
data.plot(figsize=(10,5), alpha=0.6, column='area', 
          legend=True, ax=ax, cax=cax)

The data by itself looks just like a bunch of blobs. Let's put it on a map for context

GeoDataFrame has an attribute called `.crs` that shows the coordinate system of the data which is empty (None) in our case since we are creating the data from the scratch. 

We will not go into depth about projections, which are a deep topic in and of themselves. For reference, see [this lecture](https://automating-gis-processes.github.io/2018/notebooks/L2/projections.html) from the University of Helsinki Automating GIS Processes course. In short, there are many ways to flatten the globe, each with its own benefits and issues. 

Two of the common ones we'll be using are [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator_projection), which is common in web applications, and [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System), which is the reference coordinate system used by GPS. Each has a EPSG (European Petroleum Survey Group) code associated with it. Web Mercator is EPSG 3857, and WGS84 is 4326.

[Contextily](https://github.com/darribas/contextily) is a library for creating basemaps. It pulls the data from [OpenStreetMap](https://www.openstreetmap.org), a free and excellent wiki google-maps-style service.


In [ ]:
# the data is currently in WGS84
data.crs

In [ ]:
ax = data.to_crs(epsg=3857).plot(figsize=(10,5), alpha=0.6, cmap='Set2')
# now we add a basemap. ctx finds a basemap for a background from
# an online repository
ctx.add_basemap(ax)

We can use different background styles:
![tile styles](https://raw.githubusercontent.com/darribas/contextily/master/tiles.png).

Note that some styles only contain labels or lines.

In [ ]:
sources = [i for i in dir(ctx.tile_providers) if i[0]!='_']
sources

In [ ]:
# we can get the urls for the different backgrounds using:
getattr(ctx.sources,'ST_TONER_BACKGROUND')
# we'll pass this into the add_basemap function in the url argument

In [ ]:
ax = data.to_crs(epsg=3857).plot(figsize=(10,5), alpha=0.6, cmap='Set2')
ctx.add_basemap(ax, url=getattr(ctx.sources,'ST_TONER_BACKGROUND'))

In [ ]:
# we can download background tiles as images for quicker loading (don't need to keep redownloading)
# let's use the bounds of one of the fish locations as an example
w,s,e,n = data.loc[25,'geometry'].bounds
data.loc[10,'geometry'].bounds

the function bounds2img takes coordinates and [zoom level](https://wiki.openstreetmap.org/wiki/Zoom_levels) and downloads the corresponding tiles of the map as images

In [ ]:
img, ext = ctx.bounds2img(w, s, e, n, 6, ll=True) #ll means coordinates are in lat-lon
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.imshow(img, extent=ext)
data.to_crs(epsg=3857).plot(ax=ax, cmap='Set3', alpha=0.8)
ax_bounds = data.to_crs(epsg=3857).loc[25,'geometry'].bounds
ax.set(xlim=[ax_bounds[0],ax_bounds[2]],ylim=[ax_bounds[1],ax_bounds[3]])


## Writing to a shapefile

First we'll make a directory for outputting data to. We use the `mkdir` command which makes an empty folder. The `-p` option will skip it if the directory already exists

In [ ]:
!mkdir output_data -p

In [ ]:
# let's write the first 50 rows of the shapefile to a new file
outfp = "output_data/DAMSELFISH_distributions_SELECTION.shp"

# Select first 50 rows
selection = data[0:50]

# Write those rows into a new Shapefile (the default output file format is Shapefile)
selection.to_file(outfp)

## Converting shapes to GeoDataFrames
You can use Shapely geometric objects to create a GeoDataFrame from scratch. 

In [ ]:
# Create an empty geopandas GeoDataFrame
newdata = gpd.GeoDataFrame()

# add a geometry column (necessary for shapefile)
newdata['geometry'] = None

# Let's see what we have at the moment
print(newdata)

In [ ]:
# Coordinates of the MIT main campus in Decimal Degrees
coordinates = [(-71.092562, 42.357602), ( -71.080155, 42.361553), ( -71.089817, 42.362584), (-71.094688, 42.360198)]

# Create a Shapely polygon from the coordinate-tuple list
poly = Polygon(coordinates)

# Let's see what we have
poly

In [ ]:
# Insert the polygon into 'geometry' -column at index 0
newdata.loc[0, 'geometry'] = poly
newdata

In [ ]:
newdata.loc[0, 'location'] = 'MIT main campus'
newdata

Before exporting the data it is necessary to set the coordinate reference system (projection) for the GeoDataFrame. 

We will set the crs using a function from `fiona`, another GIS library integrated with geopandas. 

In [ ]:
from fiona.crs import from_epsg

# Set the GeoDataFrame's coordinate system to WGS84 (i.e. epsg code 4326)
newdata.crs = from_epsg(4326)

# Let's see how the crs definition looks like
print(newdata.crs)

In [ ]:
outfp = "output_data/MIT_campus.shp"

# Write the data into that Shapefile
newdata.to_file(outfp)

In [ ]:
# Let's plot it
ax = newdata.to_crs(epsg=3857).plot(figsize=(10,5),alpha = 0.5, color='#FF55FF')
ctx.add_basemap(ax)
ax.set_axis_off() # remove the x-y axes

# Exercise
Find an interesting GIS dataset and:
- visualize some raw data
- ask an interesting analysis question about it:
  - intersections, sizes, quantities
  - relationships
  - e.g. which latitudes contain the most endangered species? what countries have the most ports per km of coastline?
- Visualize some of your analysis

As per usual, we'll ask a few volunteers to present their results.

Here are some resources to look for GIS datasets:
- Cambridge, MA GIS data: http://cambridgegis.github.io/gisdata.html
- Free GIS data: https://freegisdata.rtwilson.com/
- Data.gov: https://www.data.gov/